# Importing Libraries

In [154]:
import pandas as pd
import streamlit as st
import plotly.express as pt
import altair


# Analyzing

In [277]:
df = pd.read_csv("vehicles_us.csv")
df.head()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28


In [263]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


In [265]:
df.columns

Index(['price', 'model_year', 'model', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'type', 'paint_color', 'is_4wd',
       'date_posted', 'days_listed'],
      dtype='object')

In [267]:
df.isna().sum()

price               0
model_year       3619
model               0
condition           0
cylinders        5260
fuel                0
odometer         7892
transmission        0
type                0
paint_color      9267
is_4wd          25953
date_posted         0
days_listed         0
dtype: int64

In [185]:
df.duplicated().sum()

0

# Arranging Types and Missing Values

* `price` was changed to a float
* `model_year`  changed to be an string. Also, filled missing values with 'Na'.
* `cylinders` is categorical and should then be a string type. Also, filling missing values with 'Na'.
* `odometer` has missing values. Filled using the mean and changed to integer.
* `paint_color` had missing values. Filled with "na"
* Changed `date_posted` to a datetime type.

In [279]:
df['price'] = df['price'].astype('float')
df['model_year'] = df['model_year'].fillna(df['model_year'].mean()).astype('int')
df['cylinders'] = df['cylinders'].fillna('Na').astype('str')
df['odometer'] = df['odometer'].fillna(df['odometer'].mean()).astype('int')
df['paint_color'] = df['paint_color'].fillna('Na')
df['date_posted'] = pd.to_datetime(df['date_posted'], format='%Y-%m-%d')

`is_4wd` has an unusual amount of missing values (around 50%).
Investigating if 'NaN' stands for 2 wheel drive (Or simply "Not 4wd")

In [212]:
df['is_4wd'].unique()

array([1, 0])

Appears so. Replacing 'NaN' values in `is_4wd` with 0 and changing column to integer type

In [292]:
df['is_4wd'] = df['is_4wd'].fillna(0)
df['is_4wd'] = df['is_4wd'].astype('int')

In [196]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   price         51525 non-null  float64       
 1   model_year    51525 non-null  int64         
 2   model         51525 non-null  object        
 3   condition     51525 non-null  object        
 4   cylinders     51525 non-null  object        
 5   fuel          51525 non-null  object        
 6   odometer      51525 non-null  int64         
 7   transmission  51525 non-null  object        
 8   type          51525 non-null  object        
 9   paint_color   51525 non-null  object        
 10  is_4wd        51525 non-null  int64         
 11  date_posted   51525 non-null  datetime64[ns]
 12  days_listed   51525 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(4), object(7)
memory usage: 5.1+ MB


In [198]:
df.head()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400.0,2011,bmw x5,good,6.0,gas,145000,automatic,SUV,Na,1,2018-06-23,19
1,25500.0,0,ford f-150,good,6.0,gas,88705,automatic,pickup,white,1,2018-10-19,50
2,5500.0,2013,hyundai sonata,like new,4.0,gas,110000,automatic,sedan,red,0,2019-02-07,79
3,1500.0,2003,ford f-150,fair,8.0,gas,115553,automatic,pickup,Na,0,2019-03-22,9
4,14900.0,2017,chrysler 200,excellent,4.0,gas,80903,automatic,sedan,black,0,2019-04-02,28


In [200]:
df.isna().sum()

price           0
model_year      0
model           0
condition       0
cylinders       0
fuel            0
odometer        0
transmission    0
type            0
paint_color     0
is_4wd          0
date_posted     0
days_listed     0
dtype: int64

All missing values have been filled

# Setting for Streamlit

In [271]:
make_choice = df['model'].unique()
make_choice

array(['bmw x5', 'ford f-150', 'hyundai sonata', 'chrysler 200',
       'chrysler 300', 'toyota camry', 'honda pilot', 'kia sorento',
       'chevrolet silverado 1500', 'honda accord', 'ram 1500',
       'gmc yukon', 'jeep cherokee', 'chevrolet traverse',
       'hyundai elantra', 'chevrolet tahoe', 'toyota rav4',
       'chevrolet silverado', 'jeep wrangler', 'chevrolet malibu',
       'ford fusion se', 'chevrolet impala', 'chevrolet corvette',
       'jeep liberty', 'toyota camry le', 'nissan altima',
       'subaru outback', 'toyota highlander', 'dodge charger',
       'toyota tacoma', 'chevrolet equinox', 'nissan rogue',
       'mercedes-benz benze sprinter 2500', 'honda cr-v',
       'jeep grand cherokee', 'toyota 4runner', 'ford focus',
       'honda civic', 'kia soul', 'chevrolet colorado',
       'ford f150 supercrew cab xlt', 'chevrolet camaro lt coupe 2d',
       'chevrolet cruze', 'ford mustang', 'chevrolet silverado 3500hd',
       'nissan frontier crew cab sv', 'subaru imp

**Need to split make with model**

In [281]:
df['model_year'].min()

1908

In [285]:
df['model_year'].max()

2019

In [287]:
min_year, max_year = df['model_year'].min(), df['model_year'].max()

In [296]:
df['fuel'].unique()

array(['gas', 'diesel', 'other', 'hybrid', 'electric'], dtype=object)

In [298]:
df['transmission'].unique()

array(['automatic', 'manual', 'other'], dtype=object)

In [300]:
df['car_age'] = 2025 - df['model_year']

In [302]:
df.head()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,car_age
0,9400.0,2011,bmw x5,good,6.0,gas,145000,automatic,SUV,Na,1,2018-06-23,19,14
1,25500.0,2009,ford f-150,good,6.0,gas,88705,automatic,pickup,white,1,2018-10-19,50,16
2,5500.0,2013,hyundai sonata,like new,4.0,gas,110000,automatic,sedan,red,0,2019-02-07,79,12
3,1500.0,2003,ford f-150,fair,8.0,gas,115553,automatic,pickup,Na,0,2019-03-22,9,22
4,14900.0,2017,chrysler 200,excellent,4.0,gas,80903,automatic,sedan,black,0,2019-04-02,28,8


In [306]:
def age_category(x):
    if x<5: return '<5'
    elif x>=5 and x<10: return '5-10'
    elif x>=10 and x<20: return '10-20'
    else: return '>20'

In [308]:
df['age_category'] = df['car_age'].apply(age_category)

In [312]:
df.head()

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,car_age,age_category
0,9400.0,2011,bmw x5,good,6.0,gas,145000,automatic,SUV,Na,1,2018-06-23,19,14,10-20
1,25500.0,2009,ford f-150,good,6.0,gas,88705,automatic,pickup,white,1,2018-10-19,50,16,10-20
2,5500.0,2013,hyundai sonata,like new,4.0,gas,110000,automatic,sedan,red,0,2019-02-07,79,12,10-20
3,1500.0,2003,ford f-150,fair,8.0,gas,115553,automatic,pickup,Na,0,2019-03-22,9,22,>20
4,14900.0,2017,chrysler 200,excellent,4.0,gas,80903,automatic,sedan,black,0,2019-04-02,28,8,5-10
